# Sentiment Analysis 

_Natural Langauge Processing Nanodegree Program_

---



## Step 5: Switching gears - RNNs

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

In [1]:
from tensorflow.keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 8000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

17473536/17464789 [==============================] - 0s 0us/step
Loaded dataset with 25000 training samples, 25000 test samples


In [2]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 4, 2, 716, 4, 65, 7, 4, 689, 4367, 6308, 2343, 4804, 2, 2, 5270, 2, 2315, 2, 2, 2, 2, 4, 2, 628, 7685, 37, 9, 150, 4, 2, 4069, 11, 2909, 4, 2, 847, 313, 6, 176, 2, 9, 6202, 138, 9, 4434, 19, 4, 96, 183, 26, 4, 192, 15, 27, 5842, 799, 7101, 2, 588, 84, 11, 4, 3231, 152, 339, 5206, 42, 4869, 2, 6293, 345, 4804, 2, 142, 43, 218, 208, 54, 29, 853, 659, 46, 4, 882, 183, 80, 115, 30, 4, 172, 174, 10, 10, 1001, 398, 1001, 1055, 526, 34, 3717, 2, 5262, 2, 17, 4, 6706, 1094, 871, 64, 85, 22, 2030, 1109, 38, 230, 9, 4, 4324, 2, 251, 5056, 1034, 195, 301, 14, 16, 31, 7, 4, 2, 8, 783, 2, 33, 4, 2945, 103, 465, 2, 42, 845, 45, 446, 11, 1895, 19, 184, 76, 32, 4, 5310, 207, 110, 13, 197, 4, 2, 16, 601, 964, 2152, 595, 13, 258, 4, 1730, 66, 338, 55, 5312, 4, 550, 728, 65, 1196, 8, 1839, 61, 1546, 42, 2, 61, 602, 120, 45, 7304, 6, 320, 786, 99, 196, 2, 786, 5936, 4, 225, 4, 373, 1009, 33, 4, 130, 63, 69, 72, 1104, 46, 1292, 225, 14, 66, 194, 2, 1703, 56, 8, 803, 1004, 6, 2, 155, 11, 

Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

In [3]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

1654784/1641221 [==============================] - 0s 0us/step
--- Review (with words) ---
['the', 'of', 'and', 'local', 'of', 'their', 'br', 'of', 'attention', 'widow', 'alexandre', 'captures', 'parties', 'and', 'and', 'lands', 'and', 'excitement', 'and', 'and', 'and', 'and', 'of', 'and', 'english', 'crushed', 'like', 'it', 'years', 'of', 'and', 'unintentional', 'this', 'hitchcock', 'of', 'and', 'learn', 'everyone', 'is', 'quite', 'and', 'it', 'arab', 'such', 'it', 'bonus', 'film', 'of', 'too', 'seems', 'he', 'of', 'enough', 'for', 'be', 'blatantly', 'editing', 'fishburne', 'and', 'please', 'great', 'this', 'of', 'shoots', 'thing', '3', 'rooney', "it's", 'mentioning', 'and', 'carmen', 'given', 'parties', 'and', 'back', 'out', 'interesting', 'times', 'no', 'all', 'average', 'talking', 'some', 'of', 'nor', 'seems', 'into', 'best', 'at', 'of', 'every', 'cast', 'i', 'i', 'inside', 'keep', 'inside', 'large', 'viewer', 'who', 'obscure', 'and', 'sopranos', 'and', 'movie', 'of', 'routines', '

In [4]:
max_len=None
min_len=None 
for i in range(len(X_train)):
  if (max_len == None) or (len(X_train[i]) > max_len):
    max_len=len(X_train[i])
  elif (min_len == None) or (len(X_train[i]) < min_len):
    min_len=len(X_train[i])
print(max_len)
print(min_len)
  

2494
11


Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:
The maximum number of words is `2494`
the minimum number of words is `11`

...


### TODO: Pad sequences

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to `2494`.

In [5]:
 
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test
X_train=pad_sequences(X_train,maxlen=max_words,padding='post',truncating='post')
X_test=pad_sequences(X_test,maxlen=max_words,padding='post',truncating='post')

### TODO: Design an RNN model for sentiment analysis

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

In [6]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [7]:


# TODO: Design your model

model =Sequential([
    Embedding(vocabulary_size, 100, input_length=max_words),
    LSTM(200),
    Dense(1,activation='sigmoid')
])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          800000    
                                                                 
 lstm (LSTM)                 (None, 200)               240800    
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 1,041,001
Trainable params: 1,041,001
Non-trainable params: 0
_________________________________________________________________
None


#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:
The architecture i used is as follows:


*   Embedding Layer to covert the input sequence to vectors.
*   Averagepooling1D instead of the flatten.
*   LSTM Cell 
*   Two Dense layers. 

The number of trainable params is `435,457`





...

### TODO: Train and evaluate your model

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

In [8]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

> **Tip**: You can split off a small portion of the training set to be used for validation during training. This will help monitor the training process and identify potential overfitting. You can supply a validation set to `model.fit()` using its `validation_data` parameter, or just specify `validation_split` - a fraction of the training data for Keras to set aside for this purpose (typically 5-10%). Validation metrics are evaluated once at the end of each epoch.

In [ ]:
X_train[:5]

array([[   1,   14,   22, ...,    0,    0,    0],
       [   1,  194, 1153, ...,    0,    0,    0],
       [   1,   14,   47, ...,    0,    0,    0],
       [   1,    4,    2, ...,    0,    0,    0],
       [   1,  249, 1323, ...,    0,    0,    0]], dtype=int32)

In [ ]:
# Validation_size=int(0.1*len(X_train))

In [9]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 128
num_epochs = 5
X_vald=X_train[:batch_size]
X_train=X_train[batch_size:]
y_vald=y_train[:batch_size]
y_train=y_train[batch_size:]
print('the training size---data {},--labels {}'.format(len(X_train),len(y_train)))
print('the validation size---data {},--labels {}'.format(len(X_vald),len(y_vald)))
# TODO: Train your model


the training size---data 24872,--labels 24872
the validation size---data 128,--labels 128


In [10]:
model.fit(X_train, y_train,
          validation_data=(X_vald, y_vald),
          batch_size=batch_size, epochs=num_epochs)

Epoch 1/5
195/195 [==============================] - 36s 155ms/step - loss: 0.6931 - accuracy: 0.5087 - val_loss: 0.6948 - val_accuracy: 0.4219
Epoch 2/5
195/195 [==============================] - 30s 152ms/step - loss: 0.7010 - accuracy: 0.5111 - val_loss: 0.6990 - val_accuracy: 0.4375
Epoch 3/5
195/195 [==============================] - 30s 152ms/step - loss: 0.6746 - accuracy: 0.5294 - val_loss: 0.7102 - val_accuracy: 0.5703
Epoch 4/5
195/195 [==============================] - 30s 153ms/step - loss: 0.6504 - accuracy: 0.5371 - val_loss: 0.7772 - val_accuracy: 0.4297
Epoch 5/5
195/195 [==============================] - 30s 152ms/step - loss: 0.6437 - accuracy: 0.5432 - val_loss: 0.7628 - val_accuracy: 0.4375


# Training Notes 
After training the first model with the max_len as the maximum number of woords in the training data,the batch size is 128 and using average pooling and two dense layers, The results came out to be very poor on both the training and validation data. I folloed the architechture in this [paper](https://ieeexplore.ieee.org/abstract/document/9303573) to enhance the the model's performance. 

In [12]:
# Save your model, so that you can quickly load it in future (and perhaps resume training)
model_file = "rnn_model.h5"  # HDF5 file
model.save( model_file)

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [13]:
# Evaluate your model on the test set
scores = model.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Test accuracy: 0.5019199848175049
